<a href="https://colab.research.google.com/github/bayesianempirimancer/pyDMBD/blob/main/dmbd_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a example of how to use the Dynamic Markov Blanket Discovery algorithm.  The first thing you need is a properly structured data tensor:

    data = torch.randn(T, batch_num, obs_num, obs_dim)

T is the length of the time series
batch_num is the number of batches
obs_num is the number of particles/pixels/locations etc.
obs_dim is the dimension of the observables associated with each 'particle'

For example, in the Newtons cradle example there are 5 particles so obs_num is 5 and each particle has a x and y position and a x and y velocity so obs_dim is 4.  This is put into the input to the DMBD algorithm by letting obs_shape = (5,4).  


We also have to specify the number of latent and role dimension associated with the environment, boundary, and object.  

    role_dims = (8,4,8)
    hidden_dims = (4,2,4)

Optionally you can specify a batch_shape if you want to fit multiple models in parallel, i.e. batch_shape = (10,) to fit 10 models.  

Finally, regression_dim and control_dim can be used when there is a known control signal/driving force to the latent dynamical system or a tensor of regressors that influence the observations directly.  In the absense of regressors or control signals we set these to be either 0 or -1, where -1 indicates that there is no baseline or offset.  I usually set control_dim = 0 and try out both 0 and -1 for the regression_dim.  

In [ ]:
import torch
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from DynamicMarkovBlanketDiscovery import *


# Simulate Newton's Cradle
from simulations.NewtonsCradle import NewtonsCradle
dmodel = NewtonsCradle(n_balls=5,ball_size=0.2,Tmax=500,batch_size=40,g=1,leak=0.05/8,dt=0.05,include_string=False)
data_0 = dmodel.generate_data('random')[0][0::5]
data_1 = dmodel.generate_data('1 ball object')[0][0::5]
data_2 = dmodel.generate_data('2 ball object')[0][0::5]
data = torch.cat((data_0,data_1,data_2),dim=1)
dy = torch.zeros(2)
dy[1] = 1.0
data = data + dy
v_data = torch.diff(data,dim=0)
v_data = v_data/v_data.std()
data = torch.cat((data[1:],v_data),dim=-1)

# Initialize the model
model = DMBD(obs_shape=data.shape[-2:],role_dims=(8,4,8),hidden_dims=(4,2,4),regression_dim = 0, control_dim=0)




ModuleNotFoundError: ignored